In [2]:
from newspaper import Article
import random 
import string
import nltk.data
import nltk
#this is what the tutorial used
#from sklearn.feature_extraction.text import CountVectorizer
#using TF-IDF vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
#nltk.download('punkt')
#This line was not successful. Instead I imported nltk from the python interpreter and then pulled up the nltk download window to manually download punkt. This required a lot of troubleshooting before I finally got it including trying to get a zip folder of punkt from the Internet and getting its functionality manually. 

In [4]:
#Modified the demo's strategy of just using one article and instead found multiple articles from the web for the chatbot to use
#The links, articles, and blog posts that this chatbot uses to give tips are not my own. They come from a simple Google search asking for articles on acing coding interviews. 
#Links used: 
#https://www.interviewcake.com/coding-interview-tips
#https://www.freecodecamp.org/news/coding-interviews-for-dummies-5e048933b82b/'
#https://medium.com/@andreimargeloiu/the-definitive-guide-to-the-coding-interview-2704d166664c
#https://www.geeksforgeeks.org/interview-preparation/

urls = ['https://www.interviewcake.com/coding-interview-tips', 'https://www.freecodecamp.org/news/coding-interviews-for-dummies-5e048933b82b/', 'https://medium.com/@andreimargeloiu/the-definitive-guide-to-the-coding-interview-2704d166664c', 'https://www.geeksforgeeks.org/interview-preparation/']
corpus = []

for url in urls:
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    corpus.append(article.text)

#make all articles one single corpus string
corpus = ' '.join(corpus)


In [5]:
#Printing the text of the article
print(corpus)

Communicate.

Once you get into the coding questions, communication is key. A candidate who needs some help along the way but communicates clearly can be even better than a candidate who breezes through the question.

Understand what kind of problem it is. There are two types of problems:

Coding. The interviewer wants to see you write clean, efficient code for a problem. Chitchat. The interviewer just wants you to talk about something. These questions are often either (1) high-level system design ("How would you build a Twitter clone?") or (2) trivia ("What is hoisting in Javascript?"). Sometimes the trivia is a lead-in for a "real" question e.g., "How quickly can we sort a list of integers? Good, now suppose instead of integers we had . . ."

If you start writing code and the interviewer just wanted a quick chitchat answer before moving on to the "real" question, they'll get frustrated. Just ask, "Should we write code for this?"

Make it feel like you're on a team. The interviewer wa

In [6]:
#Tokenization, this gives us a list of sentences
text = corpus
sentence_list = nltk.tokenize.sent_tokenize(text)

In [7]:
#Look at the sentences
print(sentence_list)

['Communicate.', 'Once you get into the coding questions, communication is key.', 'A candidate who needs some help along the way but communicates clearly can be even better than a candidate who breezes through the question.', 'Understand what kind of problem it is.', 'There are two types of problems:\n\nCoding.', 'The interviewer wants to see you write clean, efficient code for a problem.', 'Chitchat.', 'The interviewer just wants you to talk about something.', 'These questions are often either (1) high-level system design ("How would you build a Twitter clone?")', 'or (2) trivia ("What is hoisting in Javascript?").', 'Sometimes the trivia is a lead-in for a "real" question e.g., "How quickly can we sort a list of integers?', 'Good, now suppose instead of integers we had .', '.', '."', 'If you start writing code and the interviewer just wanted a quick chitchat answer before moving on to the "real" question, they\'ll get frustrated.', 'Just ask, "Should we write code for this?"', "Make 

In [8]:
#Random Greeting Function 
def greeting(text):
    text = text.lower() 
    #Bot's greeting
    bot_greetings = ['Hello!', "Howdy!", "Hey there!", "Hi!"]
    #User's greeting
    user_greetings = ['hi', 'hey', 'hello', 'greetings', 'wassup']
    
    for word in text.split():
        if word in user_greetings: 
            return random.choice(bot_greetings)
            
        

In [9]:
#helper function to find index of highest value in list
def index_sort(list_to_sort):
    length = len(list_to_sort)
    list_index = list(range(0, length))
    x = list_to_sort
    for i in range(length): 
        for j in range(length): 
            if(x[list_index[i]] > x[list_index[j]]):
                #swap them 
                temp = list_index[i]
                list_index[i] = list_index[j]
                list_index[j] = temp
    return list_index

In [10]:
#Bot's response 
def bot_response(user_input):
    user_input = user_input.lower()
    #appending user's input to the sentence bank
    sentence_list.append(user_input)
    bot_response = ''
    #getting a count matrix from the sentence list
    #cm = CountVectorizer().fit_transform(sentence_list)
    #use TF-IDF vectorizer instead
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentence_list)
    #OLD: last sentence (the user's input) and comparing it to everything in the count matrix
    #similarity_scores = cosine_similarity(cm[-1], cm)
    #NEW: use tfidf
    similarity_scores = cosine_similarity(tfidf_matrix[-1], tfidf_matrix)
    #reduces dimensionality of the similarity scores
    similarity_scores_list = similarity_scores.flatten()
    index = index_sort(similarity_scores_list)
    index = index[1:]
    #used to see if there is a response back to the user
    response_flag = 0
    
    #variable to say if we found two or less similar sentences, will return the top one 
    temp = 0
    for i in range(len(index)):
        if similarity_scores_list[index[i]] > 0.0: #highest score 
            bot_response += ' ' + sentence_list[index[i]]
            #tells us that we found a response
            response_flag = 1
            #tells us how many scores above 0
            temp += 1
        if temp > 2:
            break
    
    if response_flag == 0: 
        bot_response += ' ' + 'I\'m sorry, I don\'t understand.'
    
    #remove the user input after done with it 
    sentence_list.remove(user_input)
    
    return bot_response 
            

In [ ]:
#Chat
print('Prep Bot: I am your interview prep bot or Prep Bot for short to help with any doubts, worries, or questions you might have about your future interviews. If you want to quit, type \'quit\'')

quit_list = ['quit', 'bye', 'exit', 'break', 'see you later', 'thank you']
user_input = input()
#print('You: ' + user_input)

while(True):
    if user_input.lower() in quit_list: 
        print('Prep Bot: Chat with you later!')
        break 
    else: 
        if greeting(user_input) != None: 
            print('Prep Bot: ' + greeting(user_input))
        else:
            #if not a greeting or exit request, find the bot_response to the user input
            print('Prep Bot: ' + bot_response(user_input))
    user_input = input()
    #print('You: ' + user_input)

Prep Bot: I am your interview prep bot or Prep Bot for short to help with any doubts, worries, or questions you might have about your future interviews. If you want to quit, type 'quit'


 hi


Prep Bot: Hello!


 behavioral interview tips


Prep Bot:  General tips

Always validate input first. Practical tips for coding questions

This section dives deep into practical tips for specific topics of algorithms and data structures, which appear frequently in coding questions. There is no tips nor tricks to get that job.
